In [19]:
import pandas as pd
import ast
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from collections import defaultdict

In [13]:
df = pd.read_csv('../noFluffJobs/no_fluffs.csv')
df = df.drop('Unnamed: 0', axis=1)


def filter_tags(str_tag_list):
    tag_list = ast.literal_eval(str_tag_list)
    return [x.strip() for x in tag_list]


df['tag'] = df['Tags'].apply(filter_tags)
df.head()

,Role,Tags,Additional,tag
0,React Frontend Developer,"[' React', ' JavaScript', ' CSS', ' Git', ' RE...","{' Job profile ': ' Mainly new features ', ' S...","[React, JavaScript, CSS, Git, REST, html5, nod..."
1,C++ Engineer,"[' C++', ' Linux', ' STL', ' Boost', ' unit te...","{' Job profile ': ' Mainly new features ', ' S...","[C++, Linux, STL, Boost, unit test, Jenkins, d..."
2,Front End Developer,"[' Angular', ' JavaScript', ' CSS', ' HTML', '...","{' Job profile ': ' Mainly new features ', ' S...","[Angular, JavaScript, CSS, HTML, React, Design..."
3,Frontend Developer,"[' JavaScript', ' jQuery', ' Angular', ' CSS',...","{' Job profile ': ' Mainly new features ', ' S...","[JavaScript, jQuery, Angular, CSS, HTML, redux..."
4,Python Developer,"[' Python', ' Django', ' PostgreSQL', ' experi...","{' Start ': ' 2020-05-01 ', ' Contract duratio...","[Python, Django, PostgreSQL, experience in dev..."


In [15]:
df['Additional'].iloc[300]

"{' Start ': ' ASAP ', ' Contract duration ': ' Permanent contract ', ' Paid holiday ': ' B2B - No  +1 ', ' Part time work ': ' No ', ' Remote possible ': ' Yes (1 day a week) ', ' Flexible hours ': ' No ', ' Travel involved ': ' No ', ' Freedom to choose tools ': ' Yes '}"

## Create the adjacency matrix

See which tags are coocurring

In [23]:
tag_dict = defaultdict(list)
all_tags = df['tag'].tolist()

flat_tags = [tag for tag_list in all_tags for tag in tag_list]
flat_tag_dict = {tag: i for i, tag in enumerate(flat_tags)}
tag_matrix = np.zeros(shape=(len(flat_tags), len(flat_tags)), dtype=np.uint8)

for tag_list in all_tags:
    for i in range(len(tag_list)):
        for j in range(len(tag_list)):
            if i == j: 
                continue
            i_tag = flat_tag_dict[tag_list[i]]
            j_tag = flat_tag_dict[tag_list[j]]
            tag_matrix[i_tag, j_tag] += 1
            tag_matrix[j_tag, i_tag] += 1

In [26]:
metrics_matrix = 1.0/tag_matrix.astype(np.float16)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


In [ ]:
nca = make_pipeline(StandardScaler(),
                    NeighborhoodComponentsAnalysis(n_components=2,
                                                   random_state=random_state))


model.fit(X_train, y_train)

# Fit a nearest neighbor classifier on the embedded training set
knn.fit(model.transform(X_train), y_train)